In [1]:
import pandas as pd
import numpy as np
from typing import Union, Optional

df = pd.read_csv('c:/Users/admin\OneDrive/Dokumente/Studium UZH/Master/5. Semester/Quantitative Portfolio Management with Python/Assingments/1. Assingment/2. Werkstatt/qpmwp-course/data/msci_country_indices.csv',
                 index_col=0, header=0, parse_dates=True, date_format='%d-%m-%Y')
#df = df.tail(2).iloc[:, :2]
print(df)

                  AT        AU        BE        CA        CH        DE  \
Index                                                                    
1999-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1999-01-04  0.010057  0.009080  0.042147  0.013070  0.035885  0.052249   
1999-01-05  0.013661 -0.010048  0.020162  0.021940  0.012016  0.001444   
1999-01-06  0.000000  0.015264 -0.000078  0.027640  0.015335  0.036205   
1999-01-07  0.004104  0.016564 -0.016877 -0.003480 -0.011902 -0.020187   
...              ...       ...       ...       ...       ...       ...   
2023-04-12 -0.002488  0.004714 -0.012072  0.001708  0.000017  0.002662   
2023-04-13  0.001297 -0.003806 -0.000049  0.004728  0.000913  0.002400   
2023-04-14  0.009533  0.004829  0.007622  0.001950  0.008387  0.005797   
2023-04-17 -0.005665  0.003404  0.004917  0.003392 -0.001485 -0.002086   
2023-04-18  0.004836 -0.003262 -0.007127  0.002161  0.004373  0.006089   

                  DK        ES       

In [2]:
def mean_ewm(X: Union[pd.DataFrame, np.ndarray, object],
                    scalefactor: Union[float, int] = 1,
                    span: Union[float, int] = 10,
                    reverse: bool = True, # False = starting from the top and true = starting from the bottom.
                    attribute: Optional[str] = None) -> Union[pd.Series, np.ndarray]:

    if not isinstance(X, pd.DataFrame):
        if attribute is not None and hasattr(X, attribute) and isinstance(getattr(X, attribute), pd.DataFrame):
        # 1.) "hasattr(X, attribute)" checks whether the object X has an attribute with the name stored in the variable "attribute".
        # 2.) "isinstance(getattr(X, attribute), pd.DataFrame)" checks whether that the attribute of X is actually a pandas DataFrame, and not something else (like a string or list).
            X = X.get_return_series()
        else:
            raise ValueError("If X is a BacktestData object, an 'attribute' must be specified.")

    denominators_dict = {}
    numerators_dict = {}
    mu_dict = {}
    mu = []
    alpha = 2 / (span + 1)

    for col in X.columns:
        series = X[col].values
        if reverse:
            series = series[::-1]

        denominators = []
        numerators = []

        for t in range(len(series)):
            value = series[t]
            if pd.isna(value):
                continue  # "pd.isna(value)"" checks if the current value is missing (i.e., NaN, None, or pd.NA). If yes, "continue" tells Python to skip the rest of the loop body and go to the next iteration.
            weight = alpha * (1 - alpha) ** t
            numerators.append(weight * value)
            denominators.append(weight)

        # Store each list under the corresponding column name
        denominators_dict[col] = denominators
        numerators_dict[col] = numerators

    for col in X.columns:
        # print(f"Weights for {col}: {denominators_dict[col]}")
        # print(f"Weights * Value for {col}: {numerators_dict[col]}")
        denom_sum = np.sum(denominators_dict[col])
        if denom_sum == 0:
            mu = np.nan
        else:
            mu = (np.sum(numerators_dict[col]) / denom_sum) * scalefactor

        mu_dict[col] = mu

    return pd.Series(mu_dict, dtype = "float64")

In [3]:
mean_ewm(df, scalefactor=1, span=10, reverse=True, attribute = "market_data")

AT    0.002798
AU    0.001788
BE    0.000074
CA    0.002864
CH    0.002667
DE    0.002657
DK    0.006635
ES    0.002692
FI   -0.000141
FR    0.003251
GB    0.003051
GR    0.005125
HK    0.001514
IE    0.005498
IL    0.000664
IT    0.003213
JP    0.003874
NL    0.000168
NO    0.003108
NZ   -0.000819
PT    0.000054
SE    0.003743
SG    0.000579
US    0.001963
dtype: float64